In [2]:
import os
import pandas as pd
import numpy  as np
from predictPy import Analisis_Predictivo
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler


<p style = "color: blue; font-size :40px;text-align:center;font-family:Times New Roman"> Tarea 4


# Ejercicio 1: [30 puntos] En este ejercicio vamos a usar la tabla de datos `potabilidad_v2.csv`, que contiene diferentes métricas de calidad del agua para cuerpos de agua distintos. Para esta tabla, las 9 primeras columnas corresponden a las variables predictoras y la variable `Potability` es la variable a predecir, la cual indica si el agua es segura para el consumo humano o no.

Realice lo siguiente:

In [53]:
datos = pd.read_csv('../datos/potabilidad_v2.csv',delimiter=',',decimal=".", index_col =0 )

print(datos.info(), "\n")
datos.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2011 entries, 4 to 3272
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2011 non-null   float64
 1   Hardness         2011 non-null   float64
 2   Solids           2011 non-null   float64
 3   Chloramines      2011 non-null   float64
 4   Sulfate          2011 non-null   float64
 5   Conductivity     2011 non-null   float64
 6   Organic_carbon   2011 non-null   float64
 7   Trihalomethanes  2011 non-null   float64
 8   Turbidity        2011 non-null   float64
 9   Potability       2011 non-null   object 
dtypes: float64(9), object(1)
memory usage: 172.8+ KB
None 



,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
4,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,No
5,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,No
6,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,No
7,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,No
8,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,No


## 1. Use Máquinas de Soporte Vectorial en Python para generar un modelo predictivo para la tabla `potabilidad_v2.csv` usando el 75 % de los datos para la tabla aprendizaje y un 25 % para la tabla testing. Obtenga los índices de precisión e interprete los resultados. Utilice los parámetros por defecto.

In [54]:
instancia_svm = SVC(random_state = 0)

analisis = Analisis_Predictivo(datos,predecir= "Potability",modelo=instancia_svm, train_size= 0.75, random_state = 0)

resultados = analisis.fit_predict_resultados()



Matriz de Confusión:
[[281  28]
 [117  77]]

Precisión Global:
0.7117296222664016

Error Global:
0.2882703777335984

Precisión por categoría:
         No        Si
0  0.909385  0.396907


## 2. Repita el ítem anterior pero intente identificar el mejor núcleo(kernel) y valor para el parámetro de regularización C. ¿Mejora la predicción?


In [55]:
instancia_svm = SVC(kernel="rbf", degree = 3, random_state = 0, C = 4)

analisis = Analisis_Predictivo(datos,predecir= "Potability",modelo=instancia_svm, train_size= 0.75, random_state = 0)

resultados = analisis.fit_predict_resultados()


Matriz de Confusión:
[[257  52]
 [ 92 102]]

Precisión Global:
0.7137176938369781

Error Global:
0.28628230616302186

Precisión por categoría:
         No        Si
0  0.831715  0.525773


+ Mejor núcleo "rbf" y mejor parámetro 4

## 3. Construya un `DataFrame` que compare el modelo construido arriba con los mejores modelos generados en tareas anteriores para la tabla `potabilidad_v2.csv`. Para esto en cada una de las filas debe aparecer un modelo predictivo y en las columnas aparezcan los índices Precisión Global, Error Global, Precisión Positiva (PP) y Precisión Negativa (PN). ¿Cuál de los modelos es mejor para estos datos? Guarde los datos de este DataFrame, ya que se irá modificando en próximas tareas.

In [56]:
### SVM

resultados.pop("Matriz de Confusión")

categorias = resultados["Precisión por categoría"]

resultados.pop("Precisión por categoría")

categorias = categorias.rename(columns={'No': 'PN', 'Si':"PP"},index = {0:'svm'})


SVM = pd.concat([pd.DataFrame([resultados], index = ['svm']), categorias], axis=1)

In [57]:
indices2 = pd.read_csv('tabla_comparativa2.csv',delimiter=',',decimal=".",index_col=0)

indices = pd.concat([SVM, indices2], axis=0)

indices

,Precisión Global,Error Global,PN,PP
svm,0.713718,0.286282,0.831715,0.525773
bosques,0.614314,0.385686,0.750809,0.396907
ADA,0.654076,0.345924,0.872483,0.336585
XG Boosting,0.660040,0.339960,0.864078,0.335052
arboles,0.689861,0.310139,0.851133,0.432990
KNN,0.644135,0.355865,0.760518,0.458763


In [58]:
indices.to_csv('tabla_comparativa3.csv', index = True)

El mejor de los modelos es `SVM` en cualquiera de los índices.

# Ejercicio 2: [45 puntos] Este conjunto de datos es originalmente del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir de forma diagnóstica si un paciente tiene diabetes o no, basándose en determinadas medidas de diagnóstico incluidas en el conjunto de datos.

Realice lo siguiente:

# 1. Cargue en Python la tabla de datos `diabetes.csv`.

In [166]:
datos = pd.read_csv('../datos/diabetes_V2.csv',delimiter=',',decimal=".",index_col=0)
print(datos.info(), "\n")
print(datos.shape)

# Convierte las variables de object a categórica
datos['genero'] = datos['genero'].astype('category')

#Convertimos a Dummy algunas de las variables predictoras
datos = pd.get_dummies(datos,columns=["genero"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 390
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   colesterol     390 non-null    int64  
 1   glucosa        390 non-null    int64  
 2   hdl_chol       390 non-null    int64  
 3   prop_col_hdl   390 non-null    float64
 4   edad           390 non-null    int64  
 5   genero         390 non-null    object 
 6   altura         390 non-null    int64  
 7   peso           390 non-null    int64  
 8   IMC            390 non-null    float64
 9   ps_sistolica   390 non-null    int64  
 10  ps_diastolica  390 non-null    int64  
 11  cintura        390 non-null    int64  
 12  caderas        390 non-null    int64  
 13  prop_cin_cad   390 non-null    float64
 14  diabetes       390 non-null    object 
dtypes: float64(3), int64(10), object(2)
memory usage: 48.8+ KB
None 

(390, 15)


# 2. Use Máquinas de Soporte Vectorial en Python (con los parámetros por defecto) para generar un modelo predictivo para la tabla `diabetes.csv` usando el 75% de los datos para la tabla aprendizaje y un 25% para la tabla testing, luego calcule para los datos de testing la matriz de confusión, la precisión global y la precisión para cada una de las dos categorías. ¿Son buenos los resultados? Explique.

In [168]:
instancia_svm = SVC(random_state = 0)

analisis = Analisis_Predictivo(datos,predecir= "diabetes",modelo=instancia_svm, train_size= 0.75, random_state = 0)

resultados = analisis.fit_predict_resultados()



Matriz de Confusión:
[[ 9 11]
 [ 0 78]]

Precisión Global:
0.8877551020408163

Error Global:
0.11224489795918369

Precisión por categoría:
   Diabetes  No diabetes
0      0.45          1.0


+ Aunque la precisión global es buena no predice bien para la variable `diabetes` que es la de interés, por lo tanto no son buenos resultados.

## 3. Repita el ítem anterior pero intente identificar el mejor núcleo(kernel) y valor para el parámetro de regularización C. ¿Mejora la predicción?

In [169]:
instancia_svm = SVC(kernel="rbf", degree = 3, random_state = 0, C = 7)

analisis = Analisis_Predictivo(datos,predecir= "diabetes",modelo=instancia_svm, train_size= 0.75, random_state = 0)

resultados = analisis.fit_predict_resultados()


Matriz de Confusión:
[[12  8]
 [ 1 77]]

Precisión Global:
0.9081632653061225

Error Global:
0.09183673469387754

Precisión por categoría:
   Diabetes  No diabetes
0       0.6     0.987179


+ Sí mejora bastante.

## 4. Con los mejores parámetros identificados en el ítem anterior realice un nuevo modelo pero haciendo selección de 6 variables. ¿Mejoran los resultados?

In [170]:
datos1 = datos[["glucosa","edad","prop_col_hdl","colesterol","peso","hdl_chol","diabetes"]]

instancia_svm = SVC(kernel="rbf", degree = 3, random_state = 0, C = 7)

analisis = Analisis_Predictivo(datos1,predecir= "diabetes",modelo=instancia_svm, train_size= 0.75, random_state = 0)

resultados1 = analisis.fit_predict_resultados()


Matriz de Confusión:
[[10 10]
 [ 0 78]]

Precisión Global:
0.8979591836734694

Error Global:
0.10204081632653061

Precisión por categoría:
   Diabetes  No diabetes
0       0.5          1.0


+ En lugar de mejorar, empeoran

## 5. Construya un `DataFrame` que compare el mejor modelo construido arriba con los mejores modelos generados en tareas anteriores para la tabla `diabetes.csv`. Para esto en cada una de las filas debe aparecer un modelo predictivo y en las columnas aparezcan los índices Precisión Global, Error Global, Precisión Positiva (PP) y Precisión Negativa (PN). ¿Cuál de los modelos es mejor para estos datos? Guarde los datos de este DataFrame, ya que se irá modificando en próximas tareas.


In [171]:
### SVM

resultados.pop("Matriz de Confusión")

categorias = resultados["Precisión por categoría"]

resultados.pop("Precisión por categoría")

categorias = categorias.rename(columns={'No diabetes': 'PN', 'diabetes':"PP"},index = {0:'svm'})


SVM = pd.concat([pd.DataFrame([resultados], index = ['svm']), categorias], axis=1)

In [172]:
indices2 = pd.read_csv('tabla_comparativa2.2.csv',delimiter=',',decimal=".",index_col=0)

indices = pd.concat([SVM, indices2], axis=0)

indices

,Precisión Global,Error Global,Diabetes,PN
svm,0.908163,0.091837,0.600000,0.987179
bosques,0.877551,0.122449,0.450000,0.987179
ADA,0.948980,0.051020,0.666667,1.000000
XG Boosting,0.867347,0.132653,0.550000,0.948718
arboles,0.887755,0.112245,0.600000,0.961538
KNN,0.877551,0.122449,0.650000,0.935897


In [173]:
indices.to_csv('tabla_comparativa2.3.csv', index = True)

## 6. Utilizando el mejor modelo construido prediga los nuevos individuos que se encuentran en el archivo `nuevos_individuos_diabetes.csv`. Recuerde que si estandarizó los datos para entrenar el modelo debe guardar valores como la media y desviación estándar para estandarizar los nuevos individuos.

In [182]:
#elimino variable a predecir
datos1 = datos.drop(['diabetes'], axis=1)

In [183]:
y_train = datos.iloc[:,13]

In [184]:
#Desviación estándar poblacional
desviaciones = datos1.std(ddof= 0)
print("Desviaciones estándar\n", desviaciones, "\n")

#Medias
medias = datos1.mean()
print("Medias\n", medias, "\n")

Desviaciones estándar
 colesterol       44.608704
glucosa          53.729172
hdl_chol         17.256902
prop_col_hdl      1.734406
edad             16.414826
altura            3.913839
peso             40.355986
IMC               6.592447
ps_sistolica     22.830202
ps_diastolica    13.480875
cintura           5.753556
caderas           5.657076
prop_cin_cad      0.073118
genero_female     0.492788
genero_male       0.492788
dtype: float64 

Medias
 colesterol       207.230769
glucosa          107.338462
hdl_chol          50.266667
prop_col_hdl       4.524615
edad              46.774359
altura            65.951282
peso             177.407692
IMC               28.775641
ps_sistolica     137.133333
ps_diastolica     83.289744
cintura           37.869231
caderas           42.992308
prop_cin_cad       0.881385
genero_female      0.584615
genero_male        0.415385
dtype: float64 



In [185]:
estandarizador = StandardScaler()


datos2 = datos1.copy()

datos2 = pd.DataFrame(estandarizador.fit_transform(datos2), columns= datos1.columns)
datos2

,colesterol,glucosa,hdl_chol,prop_col_hdl,edad,altura,peso,IMC,ps_sistolica,ps_diastolica,cintura,caderas,prop_cin_cad,genero_female,genero_male
0,-0.319013,-0.564655,-0.073401,-0.360132,-1.692029,-1.265070,-1.447312,-0.951944,-0.838071,-0.985822,-1.020105,-0.882489,-0.565995,0.842927,-0.842927
1,-1.372619,-0.527432,-0.536983,-0.533102,-1.692029,-1.520574,-1.050840,-0.360358,-1.276087,-1.875972,-0.846299,-0.528950,-0.702760,0.842927,-0.842927
2,0.218998,-0.601879,0.216339,-0.302476,-1.631108,0.267951,0.237692,0.079539,-1.188484,-0.837464,0.370339,0.354899,0.117828,0.842927,-0.842927
3,0.420753,-0.192418,1.143504,-0.763729,-1.631108,-0.498560,-1.571209,-1.391841,-0.662865,-1.430897,-1.193910,-0.705719,-1.249818,0.842927,-0.842927
4,-0.969111,-0.304089,0.969660,-1.224982,-1.631108,1.034462,-0.902163,-1.300828,-0.662865,0.201045,-1.020105,-0.705719,-0.839524,0.842927,-0.842927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,0.443170,-0.043523,-0.363140,0.389404,2.206885,-1.776077,-1.298635,-0.542385,0.563581,0.497761,-0.498688,-0.528950,-0.018937,0.842927,-0.842927
386,0.420753,3.194941,0.100443,-0.129506,2.267806,-1.520574,0.361590,1.323387,0.300771,0.349403,0.544145,0.885209,-0.429231,0.842927,-0.842927
387,2.102039,-0.322701,3.924999,-1.109668,2.572409,-1.265070,-1.546430,-1.073295,3.542092,0.497761,-1.193910,-0.352180,-1.660112,0.842927,-0.842927
388,0.555256,1.426814,3.693208,-1.455608,2.694250,-1.265070,-1.249076,-0.724411,1.439613,-0.095672,-0.498688,-0.882489,0.528122,0.842927,-0.842927


In [201]:
datos_nuevos = pd.read_csv('../datos/nuevos_individuos_diabetes.csv',delimiter=',',decimal=".",index_col=0)



# Convierte las variables de object a categórica
datos_nuevos['genero'] = datos_nuevos['genero'].astype('category')

#Convertimos a Dummy algunas de las variables predictoras
datos_nuevos = pd.get_dummies(datos_nuevos,columns=["genero"])

#elimino variable a predecir
datos_nuevos1 = datos_nuevos.drop(['diabetes'], axis=1)


In [202]:
y_test = datos_nuevos.iloc[:,13]

In [203]:
datos_nuevos3 = datos_nuevos1.copy()

#No debemos utilizar fit
datos_nuevos3 = pd.DataFrame(estandarizador.transform(datos_nuevos3), columns= datos_nuevos1.columns)
datos_nuevos3

,colesterol,glucosa,hdl_chol,prop_col_hdl,edad,altura,peso,IMC,ps_sistolica,ps_diastolica,cintura,caderas,prop_cin_cad,genero_female,genero_male
0,-0.319013,2.617973,-1.522096,2.003790,0.744793,0.012448,0.287251,0.261566,0.125565,0.497761,0.022728,0.354899,-0.565995,0.842927,-0.842927
1,-0.274179,0.012312,-0.247244,-0.187162,0.744793,0.267951,-0.134000,-0.284514,0.563581,1.387911,0.022728,0.001360,-0.018937,0.842927,-0.842927
2,0.263833,0.086760,1.317347,-0.879042,0.744793,0.012448,-0.183559,-0.208669,0.388374,0.646120,-0.151077,-0.528950,0.664887,-1.186342,1.186342
3,1.833033,2.971599,-0.710827,1.773163,0.744793,0.523455,-0.208338,-0.466540,0.213168,-0.318210,-0.324883,-0.882489,0.938416,-1.186342,1.186342
4,0.622507,-0.099359,-0.479035,0.620031,0.805713,0.778958,0.212913,-0.193500,0.475978,1.091194,0.370339,-0.175410,0.938416,-1.186342,1.186342
5,1.564027,0.217043,0.679921,-0.014192,0.805713,-1.265070,-1.224297,-0.694073,0.125565,0.201045,-0.846299,-0.705719,-0.429231,0.842927,-0.842927
6,-1.013945,-0.322701,-0.247244,-0.590759,0.805713,-0.754063,-1.397753,-1.118802,-1.188484,-1.430897,-1.020105,-1.589568,0.801651,0.842927,-0.842927
7,2.483130,3.027434,3.345521,-0.936699,0.805713,-0.243056,-0.257897,-0.147994,-0.224848,-0.837464,0.022728,0.178130,-0.292466,0.842927,-0.842927
8,1.608862,3.027434,-0.594931,1.427223,0.805713,0.523455,1.154533,0.807645,1.614820,0.497761,1.760784,1.238748,1.075180,0.842927,-0.842927
9,-1.776128,2.152677,-1.522096,0.447061,0.805713,0.267951,0.460708,0.291904,-1.188484,-1.134180,0.717951,0.001360,1.348710,-1.186342,1.186342


In [206]:
from sklearn.metrics import confusion_matrix

def indices_general(MC, nombres = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    precision_categoria  = pd.DataFrame(MC.diagonal()/np.sum(MC,axis = 1)).T
    if nombres!=None:
        precision_categoria.columns = nombres
    return {"Matriz de Confusión":MC, 
            "Precisión Global":precision_global, 
            "Error Global":error_global, 
            "Precisión por categoría":precision_categoria}

instancia_svm = SVC(kernel="rbf", degree = 3, random_state = 0, C = 7)

instancia_svm.fit(datos2.values,y_train)

prediccion = instancia_svm.predict(datos_nuevos3.values)

labels = ["Diabetes", "No diabetes"]

MC = confusion_matrix(y_test, prediccion,labels=labels)

indices = indices_general(MC,labels)
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusión:
[[7 3]
 [0 9]]

Precisión Global:
0.8421052631578947

Error Global:
0.1578947368421053

Precisión por categoría:
   Diabetes  No diabetes
0       0.7          1.0


# Ejercicio 3: [25 puntos] Según el ejemplo de los hiperplanos visto en clase realice lo siguiente:

In [11]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.express as px

In [16]:
d = {'x': [1, 1, 1, 3, 1, 3, 1, 3, 1], 'y': [0, 0, 1, 1, 1, 2, 2, 2, 1], 
  'z': [1, 2, 2, 4, 3, 3, 1, 1, 0], 
  'Clase': ['Rojo', 'Rojo', 'Rojo', 'Rojo', 'Rojo', 'Azul', 'Azul', 'Azul', 'Azul']}
df = pd.DataFrame(data = d)
df

,x,y,z,Clase
0,1,0,1,Rojo
1,1,0,2,Rojo
2,1,1,2,Rojo
3,3,1,4,Rojo
4,1,1,3,Rojo
5,3,2,3,Azul
6,1,2,1,Azul
7,3,2,1,Azul
8,1,1,0,Azul


## 1. Escriba la regla de clasificación para el clasificador con margen máximo. Debe ser algo como lo siguiente: $w = (w_1, w_2, w_3)$ se clasifica como $Rojo$ si $ax + by + cz + d > 0$, de otra manera se clasifica como $Azul$.

Rojo si:

$$-x - y + z > 0$$

Azul si:

$$
-x -y +z +2 >0
$$

## 2. Indique la medida del margen entre el hiperplano óptimo de separación y los vectores de soporte.


In [17]:
df.insert(4, "esSoporte", [True,False,True,True,False,True,True,False,True], True)

df

,x,y,z,Clase,esSoporte
0,1,0,1,Rojo,True
1,1,0,2,Rojo,False
2,1,1,2,Rojo,True
3,3,1,4,Rojo,True
4,1,1,3,Rojo,False
5,3,2,3,Azul,True
6,1,2,1,Azul,True
7,3,2,1,Azul,False
8,1,1,0,Azul,True


In [22]:
x, y = np.meshgrid(range(5), range(5))
z_rojo = x + y
z_azul = x + y - 2
z_optimo = x + y - 1

In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x = df[(df['esSoporte'] == False) & (df['Clase'] == 'Rojo')]['x'],
    y = df[(df['esSoporte'] == False) & (df['Clase'] == 'Rojo')]['y'],
    z = df[(df['esSoporte'] == False) & (df['Clase'] == 'Rojo')]['z'],
    mode = "markers", marker = dict(color = "red"),
    name = "Rojo"
))

fig.add_trace(go.Scatter3d(
    x = df[(df['esSoporte'] == False) & (df['Clase'] == 'Azul')]['x'],
    y = df[(df['esSoporte'] == False) & (df['Clase'] == 'Azul')]['y'],
    z = df[(df['esSoporte'] == False) & (df['Clase'] == 'Azul')]['z'],
    mode = "markers", marker = dict(color = "blue"),
    name = "Azul"
))

fig.add_trace(go.Scatter3d(
    x = df[(df['esSoporte'] == True) & (df['Clase'] == 'Rojo')]['x'],
    y = df[(df['esSoporte'] == True) & (df['Clase'] == 'Rojo')]['y'],
    z = df[(df['esSoporte'] == True) & (df['Clase'] == 'Rojo')]['z'],
    mode = "markers", marker = dict(color = "pink"),
    name = "Soporte Rojo"
))

fig.add_trace(go.Scatter3d(
    x = df[(df['esSoporte'] == True) & (df['Clase'] == 'Azul')]['x'],
    y = df[(df['esSoporte'] == True) & (df['Clase'] == 'Azul')]['y'],
    z = df[(df['esSoporte'] == True) & (df['Clase'] == 'Azul')]['z'],
    mode = "markers", marker = dict(color = "aqua"),
    name = "Soporte Azul"
))

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = z_rojo,
    opacity = .7, showscale = False, 
    colorscale = np.repeat('brown', x.size, axis = 0),
    name="Margen"
))

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = z_azul,
    opacity = .7, showscale = False, 
    colorscale = np.repeat('brown', x.size, axis = 0),
    name="Margen"
))

## 3. Explique por qué un ligero movimiento de la octava observación no afectaría el hiperplano de margen máximo.

In [29]:
df.iloc[7, :]


x                3
y                2
z                1
Clase         Azul
esSoporte    False
Name: 7, dtype: object

In [30]:
# Posibles lijeros movimientos de de la octava observación
d = {'x': [2, 4, 3, 3, 3, 3], 'y': [2, 2, 3, 1, 2, 2], 'z': [1, 1, 1, 1, 2, 2]}
ocatava_observacion = pd.DataFrame(data = d)
ocatava_observacion

,x,y,z
0,2,2,1
1,4,2,1
2,3,3,1
3,3,1,1
4,3,2,2
5,3,2,2


In [31]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x = df[df['Clase'] == 'Rojo']['x'],
    y = df[df['Clase'] == 'Rojo']['y'],
    z = df[df['Clase'] == 'Rojo']['z'],
    mode = "markers", marker = dict(color = "red"),
    name = "Rojo"
))

fig.add_trace(go.Scatter3d(
    x = df[df['Clase'] == 'Azul']['x'],
    y = df[df['Clase'] == 'Azul']['y'],
    z = df[df['Clase'] == 'Azul']['z'],
    mode = "markers", marker = dict(color = "blue"),
    name = "Azul"
))

fig.add_trace(go.Scatter3d(
    x = ocatava_observacion['x'],
    y = ocatava_observacion['y'],
    z = ocatava_observacion['z'],
    mode = "markers", marker = dict(color = "aqua"),
    name = "Octava observación"
))

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = z_rojo,
    opacity = .7, showscale = False, 
    colorscale = np.repeat('red', x.size, axis = 0)
))

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = z_azul,
    opacity = .7, showscale = False, 
    colorscale = np.repeat('blue', x.size, axis = 0)
))

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = z_optimo,
    opacity = .7, showscale = False, 
    colorscale = np.repeat('green', x.size, axis = 0)
))

Observar que en ninguno de los movimientos traspasa el vector de soporte, por tanto no afecta el resultado